# Multi-objective optimisation of reservoir release scheduling
In this Notebook we will use a mathematical model to simulate some illustrative study-cases and a multi-objective optimizer to find optimal release and pumped inflows schedulings. The study cases will gradually biuld-up in complexity from a single reservoir system to a pumped reservoir system; and from a deterministic approach where forecast ensemble uncertainty is reduced a priori (=before running optimisation) to a stochastic approach where instead forecast ensemble uncertainty is explicitely considered within the optimisation process.

## 1. Import libraries
First, we need to import the necessary libraries and tools. (🚨 in order to run the code like in the box below, place the mouse pointer in the cell, then click on “run cell” button above or press shift + enter)

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from platypus import NSGAII, Problem, Real
import sys
import warnings
warnings.filterwarnings('ignore')

**Tools from the iRONS Software**

In [ ]:
from irons.Software.read_data import read_csv_data
from irons.Software.day2week2month import day2week

## 2. Study case: Single reservoir system - Determinisitc forecast
We consider a simple illustrative system of a reservoir operated to supply water to a domestic consumption node while ensuring a minimum environmental flow in the downstream river and maintaining the water level as high as possible to ensure the resource availability.

### 2.1 The reservoir model
<left><img src="../../util/images/System 1 layout.png" width = "400px"><left>
The mathematical model of the reservoir essentially consists of a water balance equation, where the storage (***s***) at a future time step (for example, at the beginning of the next week) is predicted from the storage at the current time (the beginning of the this week) by adding and subtracting the inflows and outflows that will occur during the temporal interval ahead:

***s(t+1) = s(t) + I_S(t) – E(t) – env(t) - spill(t) – Qreg_S_D(t)***

Where

***s(t)*** = reservoir storage at time-step t, in Vol (for example: ML)

***I(t)*** = reservoir inflows in the interval [t,t+1], in Vol/time (for example: ML/week). This is usually provided by a flow forecasting system or assumed by looking, for example, at historical inflow records for the relevant time of year

***E(t)*** = evaporation from the reservoir surface area in the interval [t,t+1], in Vol/time (for example: ML/week). This is calculated internally to the model, by multipling the evaporation rate for unit surface area (which depends on air temperature) by the reservoir surface area (which is derived from the storage S given that the reservoir geometry is known)

***env(t)*** = environmental compensation flow in the interval [t,t+1], in Vol/time (for example: ML/week). This is usually set to the value that was agreed upon with the environemtal regulator ([Learn more about the rational behind environmental flows](https://www.youtube.com/watch?v=cbUrrYq9BmU))

***spill(t)*** = outflow through spillways (if any) in the interval [t,t+1], in Vol/time (for example: ML/week). This is calculated internally to the model, and is equal to the excess volume with respect to the maximum reservoir capacity (so most of the time ***spill(t)*** is equal to 0 as the maximum capacity is not reached, but it occasionally is >0 so that the capacity is never exceeded)

***Qreg(t)*** = reservoir regulated release for water supply in the interval [t,t+1], in Vol/time (for example: ML/week). This is a completely free variable that the reservoir operator will need to specify

### 2.2 Inputs
#### 2.2.1 Definition of inflow and demand scenarios
Let's consider a forecast period of 30 weeks, and load the uncertain forecasts of inflows (I) and evaporation (e) for this period. The uncertainty in the forecasts, which represented by an ensemble of 25 possible time series, is reduced a priori by considering the average of the forecast ensemble.

In [ ]:
N = 30 # weeks

**Evaporation forecast**

In [ ]:
e_fore_folder_path = 'Results/' # folder containing the forecast weather data
e_fore_file = 'e_fore_corr.csv'
dates_fore_day, e_fore_day = read_csv_data(e_fore_folder_path, e_fore_file)
dates_fore, e_fore_ens, e_fore_cum_ens = day2week(dates_fore_day,e_fore_day)
e_fore = np.zeros((dates_fore.shape[0],1))
e_fore[:,0] = e_fore_ens.mean(1) # Average of the forecast ensemble

**Inflow forecast**

In [ ]:
I_fore_folder_path = 'Results/' # folder containing the forecast weather data
I_fore_file = 'I_fore_week.csv'
dates_fore, I_fore_ens = read_csv_data(I_fore_folder_path, I_fore_file)
I_fore = np.zeros((dates_fore.shape[0],1))
I_fore[:,0] = I_fore_ens.mean(1) # Average of the forecast ensemble

In [ ]:
I_fore.shape

**Demand forecast**

Due to the abscence of demand forecast information, in this study case we assume the water demand forecast as constant value equal to the historical mean value.

In [ ]:
d_fore = 70.0 # (ML/week)

#### 2.2.2 Definition of other input parameters 
Let's define other variables that are needed for the reservoir system simulation, such as the reservoir storage capacity, the environmental compensation flow, etc.

**System constraints**

In [ ]:
# Minimum environmental flow
env_min = 2 # (ML/week)

# Maximum regulated flows
Qreg_S_D_min = 0 # (ML/week) Minumum regulated release flow
Qreg_S_D_max = 250 # (ML/week) Maximum regulated release flow

# Min and max storage
s_min = 0 # (ML) Minimum active storage
s_max = 2000 # (ML) Maximum storage (=reservoir capacity)

**Initial conditions**

In [ ]:
s_ini = 1200 # (ML) Storage volume at the beginning of the simulation period

### 2.3 Two-objective optimization of the reservoir release scheduling
#### Implementation of the reservoir simulation function
Here we load the submodule that implements the reservoir simulation model (**Res_sys_sim**). Below is a diagram of the function including the mass balance equation and how the function deals with regulated flows (***Qreg***). The function allows to define both regulated inflows (***Qreg_inf***) and regulated releases (***Qreg_rel***) either as a time series, i.e. array of values for each time step, or a policy function, i.e. a function that can be used to determine the release conditional on the state of the reservoir system in the current time-step. In this Notebook we will define the regulated flows only as time series but in the next section ('Multi-objective optimisation of reservoir operating policy') we will use policy functions.

<left><img src="../../Software/res_sys_sim_diagram.png" width = "1000px"><left>

In [ ]:
from irons.Software.res_sys_sim import res_sys_sim

#### Two-objective optimization

The following code will run an automatic optimisation of the reservoir system, considering two objectives: Mean Squared Deficit (MSD) and Mean Resource Deficit (MRD).

Let's assume that, we are interested in minimising the deficit with respect to a historical water demand, that is, to minimise the objective function:

$$MSD = 1/N\sum_{t=1}^{N} [ \ max( \ 0, \ d(t)-u(t) \ ) \ ]^2 $$

where N is the length of the simulation period that we are considering, and d(t) is the water demand for each time-interval in that period, and MSD stands for Mean Squared Deficit. Notice that the function $max(0,...)$ enables us to only count the difference between demand d and release u when this is positive, that is, when the release u is smaller than the demand d, and a water shortage is indeed produced. Also, the squaring is a 'mathematical trick' to make sure that larger deficit amounts are given more weight than smaller ones. 

We are also interested in maximizing the resource availability to reduce the risk of water deficit. We measure how well this criterion is satisfied by the following objective function:

$$MRD = 1/N\sum_{t=1}^{N} [ \ s_{max} - s(t) \ ] $$

where, again, N is the length of the simulation period, s is the reservoir storage, and s_max is the maximum reservoir storage (MRD stands for Mean Resource Deficit).

To this end, we use the Python Platypus package, and the NSGAII algorithm implemented in it. For more information about these methods and tools, see [Deb et al, 2002](https://ieeexplore.ieee.org/document/996017) and the [Platypus webpage](https://platypus.readthedocs.io). The code to run the optimisation is the following:

In [ ]:
# Optimizer
pop_size = 40 # population size
num_iter = 10000 # Number of iterations

def auto_optim(vars):
    Qreg_S_D = np.zeros((N+1,1))
    # Decision vector: regulated release to meet the demand in D
    Qreg_S_D[:,0] = np.array(vars[0:N+1])
    
    Qreg = {'releases' : {'type'  : 'scheduling',
                          'input' : Qreg_S_D},
            'inflows'  : [],
            'rel_inf'  : []}

    # Reservoir system simulation
    Qenv, Qspill, Qreg_S_D, I_reg, s, E = res_sys_sim(I_fore, e_fore,
                                                      s_ini, s_min, s_max,
                                                      env_min, d_fore,
                                                      Qreg)

    # Objective functions
    SD = (np.maximum(d_fore-Qreg_S_D,0))**2 # supply deficit objective
    RD = s_max-s # resource deficit objective
    MSD = np.mean(SD[1:])
    MRD = np.mean(RD[1:])
    
    return [MSD, MRD]

problem = Problem(N+1,2) # Problem(number of optimizer variables, num of objective functions)
Real0 = Real(Qreg_S_D_min, Qreg_S_D_max) # Range of values

problem.types[:] = [Real0]*(N+1)
problem.function = auto_optim

algorithm_1 = NSGAII(problem,pop_size)
algorithm_1.run(num_iter)

#### Optimization results
Optimal release schedulings

In [ ]:
sol_optim_1 = [algorithm_1.result[i].variables for i in range(pop_size)]

Objective functions

In [ ]:
results_MSD_1 = np.array([algorithm_1.result[i].objectives[0] for i in range(pop_size)])
results_MRD_1 = np.array([algorithm_1.result[i].objectives[1] for i in range(pop_size)])

#### Plot the optimisation results
We can visualise the tradeoffs between the two objectives in one plot, called Pareto front, which displays the combination of the two objective values in correspondence to a set of optimised solutions. 

**Comment:** Please note that the figure is interactive, you can zoom in and out as well as select with the mouse the range of values (both horizontally or vertically) you would like to visualize in the figure. 

In [ ]:
# We define the figure layout
layout_1 = go.Layout(title  = 'Tradeoff between objectives (Pareto front)',
                     xaxis  = dict(showgrid=False,title = 'squared supply defict [ML^2]'),
                     yaxis  = dict(title = 'volume to maximum capacity [ML]'),
                     width  = 600, 
                     height = 600)

# We define the figure using the layout
fig_1 = go.Figure(layout = layout_1)

# We add the trace corresponding to the average objective results
fig_1.add_trace(go.Scatter(x       = results_MSD_1,
                           y       = results_MRD_1,
                           name    = 'mean',
                           mode    = 'markers',
                           marker  = dict(color='blue',
                                          size = 10)))

## 3. Study case: Single reservoir system - Ensemble forecast
Now we consider again the same reservoir system but now we account for the uncertainty in the forecasts, represented by the M members of the inflow forecast ensemble.

In [ ]:
# Number of forecast ensemble members
M = I_fore_ens.shape[1]

### 3.1 Optimization of the reservoir release scheduling
In this approach, instead of pre-filtering the uncertainty by selecting one ensemble member to optimise against, we carry over the entire ensemble into the optimisation procedure. The optimisation algorithm will therefore look for solutions that perform 'best' (on average) against all the ensemble members, instead of doing best for one specific ensemble member:
#### Two-objective optimization
The following code will run an automatic optimisation of the reservoir system, considering two objectives: Mean Squared Deficit (MSD) and Mean Resource Deficit (MRD).

Let's assume that, we are interested in minimising the deficit with respect to a historical water demand, that is, to minimise the objective function:

$$MSD = 1/M\sum_{t=1}^{M} \ [ \ 1/N\sum_{t=1}^{N} [ \ max( \ 0, \ d(t,m)-u(t) \ ) \ ]^2 \ ] $$

where M is the number of members of the inflow forecast ensemble, N is the length of the simulation period that we are considering, and d(t) is the water demand for each time-interval in that period. 

We are also interested in maximizing the resource availability to reduce the risk of water deficit. We measure how well this criterion is satisfied by the following objective function:

$$MRD = 1/M\sum_{t=1}^{M} \ [ \ 1/N\sum_{t=1}^{N} [ \ s_{max} - s(t,m) \ ] \ ] $$

where s is the reservoir storage, and s_max is the maximum reservoir storage.

In [ ]:
def auto_optim(vars):
    # Initialization of the optimization objective functions
    MSD = 0
    MRD = 0
    
    # Decision vector: regulated release to meet the demand in D
    Qreg_S_D = np.array(vars[0:(N+1)]).reshape(N+1,1)
    Qreg = {'releases' : {'type'  : 'scheduling',
                          'input' : Qreg_S_D},
            'inflows'  : [],
            'rel_inf'  : []}
    
    # Reservoir system simulation for each member of the forecast ensemble 
        
    Qenv, Qspill, Qreg_S_D, Qreg_inf, s, E = res_sys_sim(I_fore_ens,e_fore_ens,
                                                         s_ini,s_min,s_max,
                                                         env_min,d_fore,
                                                         Qreg)

    # Objective functions
    SD  = (d_fore-Qreg_S_D).clip(min=0)**2 # supply deficit objective
    RD  = s_max-s # resource deficit objective
    MSD = np.mean(np.mean(SD[1:,:],axis = 0)) # Average weekly SD for the M ensemble members
    MRD = np.mean(np.mean(RD[1:,:],axis = 0)) # Average weekly RD for the M ensemble members

    return [MSD, MRD]

problem = Problem((N+1),2)
Real0 = Real(Qreg_S_D_min, Qreg_S_D_max)

problem.types[:] = [Real0]*(N+1)
problem.function = auto_optim

algorithm_2 = NSGAII(problem,pop_size)
algorithm_2.run(num_iter) # Number of iterations

#### Optimization results
We obtain the results for the objective functions by simulating the system applying the optimized release schedulings for each ensemble member:

Optimal release schedulings

In [ ]:
sol_optim_2 = np.array([algorithm_2.result[i].variables for i in range(pop_size)])

Objective functions: mean system performance

In [ ]:
results_MSD_2 = np.array([algorithm_2.result[i].objectives[0] for i in range(pop_size)])
results_MRD_2 = np.array([algorithm_2.result[i].objectives[1] for i in range(pop_size)])

Objective functions: system performance for all the ensemble members

In [ ]:
# Initialization of the objective functions
results_MSD_all_2 = np.zeros([pop_size,M])*np.nan
results_MRD_all_2 = np.zeros([pop_size,M])*np.nan

# System simulation for each for the ensemble members:
for i in range(pop_size):
    Qreg_S_D = sol_optim_2[i,:].reshape(N+1,1)
    Qreg = {'releases' : {'type'  : 'scheduling',
                          'input' : Qreg_S_D},
            'inflows'  : [],
            'rel_inf'  : []}

    Qenv, Qspill, Qreg_S_D, Qreg_inf, s, E = res_sys_sim(I_fore_ens,e_fore_ens,
                                                         s_ini,s_min,s_max,
                                                         env_min,d_fore,
                                                         Qreg)

    SD  = (d_fore-Qreg_S_D).clip(min=0)**2 # supply deficit objective
    RD  = s_max-s # resource deficit objective
    results_MSD_all_2[i,:] = np.mean(SD[1:,:],axis = 0) # mean SD for each ensemble member (m)
    results_MRD_all_2[i,:] = np.mean(RD[1:,:],axis = 0) # mean RD for each ensemble member (m)

#### Plot the optimisation results
We can visualise the tradeoffs between the two objectives in one plot (Pareto front) including the forecast uncertainty. 

**Comment:** To represent the uncertainty of the forecasts the results of the system objectives for each of the ensemble members is represented. Each forecast ensemble member is numerated.

**Comment:** Please note that the figure is interactive, if you click on the legend you will be able to delete the selected member of the forecast ensemble, you can also zoom in and out as well as select with the mouse the range of values (both horizontally or vertically) you would like to visualize in the figure. 

In [ ]:
# We define the figure layout
layout_2 = go.Layout(title  = 'Tradeoff between objectives (Pareto front)',
                     xaxis  = dict(showgrid=False,title = 'squared supply defict [ML^2]'),
                     yaxis  = dict(title = 'volume to maximum capacity [ML]'),
                     width  = 600, 
                     height = 600)

# We define the figure using the layout
fig_2 = go.Figure(layout = layout_2)

# We add all the traces corresponding to the objective results for each ensemble member
for m in range(M):
    fig_2.add_trace(go.Scatter(x = results_MSD_all_2[:,m],
                               y = results_MRD_all_2[:,m],
                               name    = 'member '+str(m),
                               mode    = 'markers',
                               marker  = dict(color='red',
                                              opacity = 0.1,
                                              size = 10)))

# We add the trace corresponding to the average objective results
fig_2.add_trace(go.Scatter(x       = results_MSD_2,
                           y       = results_MRD_2,
                           name    = 'mean',
                           mode    = 'markers',
                           marker  = dict(color='blue',
                                          size = 10)))

## 4. Study case: Pumped single reservoir system - Ensemble forecast
We consider again the same simple illustrative system of a reservoir operated to supply water to a domestic consumption node and a seasonal inflow forecast ensemble. This time however we assume that the reservoir inflows can be augmented by pumping river abstractions into the reservoir. The decision to be made is about how much water to pump into the reservoir (***u***). So, the objective is to determine the pumping scheduling that will minimise pumping costs, while meeting the forecasted water demand (***d***) over a coming period of time (we will assume here that the release ***r*** is always set to the demand ***d***, unless physically impossible, that is, unless the reservoir storage is too low and the release must be reduced). 
<left><img src="../../util/images/System 2 layout.png" width = "500px"><left>
### 4.1 Additional inputs
#### Pumping energy cost
Here we define the energy cost per ML of pumping water from the streamflow ***R*** to the reservoir ***S*** 

In [ ]:
c_R_S = 50 # £/ML

#### Maximum regulated inflows

In [ ]:
# Maximum regulated flows
Qreg_R_S_min = 0 # (ML/week) Minumum regulated release flow
Qreg_R_S_max = 250 # (ML/week) Maximum regulated release flow

### 4.2 Optimization of the reservoir pumped inflows
#### Two-objective optimization

The following code will run an automatic optimisation of the reservoir system, considering two objectives: Mean Pumping cost (MPC) and Mean Resource Deficit (MRD):

We are interested in minimising the pumping energy cost, that is, to minimise the objective function:

$$MPC = 1/M\sum_{t=1}^{M} \ [ \ 1/N\sum_{t=1}^{N} [ \ u(t) \ * \ c \ ] \ ] $$

where M is the number of members of the inflow forecast ensemble, N is the length of the simulation period that we are considering, u(t) is the pumped inflow and c the pumping energy cost (£) per ML.

We are also interested in maximizing the resource availability to reduce the risk of water deficit. We measure how well this criterion is satisfied by the following objective function:

$$MRD = 1/M\sum_{t=1}^{M} \ [ \ 1/N\sum_{t=1}^{N} [ \ s_{max} - s(t,m) \ ] \ ] $$

where s is the reservoir storage, and s_max is the maximum reservoir storage..

In [ ]:
def auto_optim(vars):
    # Initialization of the optimization objective functions
    MPC = 0
    MRD = 0
    
    # Decision vector: regulated pumped inflows
    Qreg_R_S = np.array(vars[0:N+1]).reshape(N+1,1)
    Qreg = {'releases' : [],
            'inflows'  : {'type'  : 'scheduling',
                          'input' : Qreg_R_S},
            'rel_inf'  : []}
 
    # Reservoir system simulation for all the members of the forecast ensemble
    Qenv, Qspill, Qreg_S_D, Qreg_R_S, s, E = res_sys_sim(I_fore_ens,e_fore_ens,
                                                         s_ini,s_min,s_max,
                                                         env_min,d_fore,
                                                         Qreg)
    
    # Objective functions
    PC = Qreg_R_S * c_R_S # regulated inflows pumping cost
    RD = s_max - s # resource deficit objective
    MPC = np.mean(np.mean(PC[1:,:],axis = 0)) # Average weekly PC for the M ensemble members
    MRD = np.mean(np.mean(RD[1:,:],axis = 0)) # Average weekly RD for the M ensemble members

    return [MPC, MRD]

problem = Problem(N+1,2)
Real0 = Real(Qreg_R_S_min, Qreg_R_S_max)

problem.types[:] = [Real0]*(N+1)
problem.function = auto_optim

algorithm_3 = NSGAII(problem,pop_size)
algorithm_3.run(num_iter) # Number of iterations

#### Optimization results
We obtain the results for the system objectives by computing the system objectives for each ensemble member:

Optimal pumped inflow schedulings

In [ ]:
sol_optim_3 = np.array([algorithm_3.result[i].variables for i in range(pop_size)])

Objective functions: mean system performance

In [ ]:
results_MPC_3 = np.array([algorithm_3.result[i].objectives[0] for i in range(pop_size)])
results_MRD_3 = np.array([algorithm_3.result[i].objectives[1] for i in range(pop_size)])

Objective functions: system performance for all the ensemble members

In [ ]:
# Initialization of the objective functions
results_MPC_all_3 = np.zeros([pop_size,M])*np.nan
results_MRD_all_3 = np.zeros([pop_size,M])*np.nan

# System simulation for each for the ensemble members:
for i in range(pop_size):
    Qreg_R_S = sol_optim_3[i,:].reshape(N+1,1)
    Qreg = {'releases' : [],
            'inflows'  : {'type'  : 'scheduling',
                          'input' : Qreg_R_S},
            'rel_inf'  : []}

    Qenv, Qspill, Qreg_S_D, Qreg_R_S, s, E = res_sys_sim(I_fore_ens,e_fore_ens,
                                                         s_ini,s_min,s_max,
                                                         env_min,d_fore,
                                                         Qreg)

    PC = Qreg_R_S * c_R_S # regulated inflows pumping cost
    RD = s_max-s # resource deficit objective
    results_MPC_all_3[i,:] = np.mean(PC[1:,:],axis = 0) # mean PC for each ensemble member (m)
    results_MRD_all_3[i,:] = np.mean(RD[1:,:],axis = 0) # mean RD for each ensemble member (m)

#### Plot the optimisation results
We can visualise the tradeoffs between the two objectives in one plot (Pareto front) including the forecast uncertainty. 

**Comment:** To represent the uncertainty of the forecasts the results of the system objectives for each of the ensemble members is represented. Each forecast ensemble member is numerated.

**Comment:** Please note that the figure is interactive, if you click on the legend you will be able to delete the selected member of the forecast ensemble, you can also zoom in and out as well as select with the mouse the range of values (both horizontally or vertically) you would like to visualize in the figure. 

In [ ]:
# We define the figure layout
layout_3 = go.Layout(title  = 'Tradeoff between objectives (Pareto front)',
                     xaxis  = dict(showgrid=False,title = 'pumping cost [£/week]'),
                     yaxis  = dict(title = 'volume to maximum capacity [ML]'),
                     width  = 600, 
                     height = 600)

# We define the figure using the layout
fig_3 = go.Figure(layout = layout_3)

# We add all the traces corresponding to the objective results for each ensemble member
for m in range(M):
    fig_3.add_trace(go.Scatter(x       = results_MPC_all_3[:,m],
                               y       = results_MRD_all_3[:,m],
                               name    = 'member '+str(m),
                               mode    = 'markers',
                               marker  = dict(color='red',
                                              opacity = 0.1,
                                              size = 10)))
    
# We add the trace corresponding to the average objective results
fig_3.add_trace(go.Scatter(x       = results_MPC_3,
                           y       = results_MRD_3,
                           name    = 'mean',
                           mode    = 'markers',
                           marker  = dict(color='blue',
                                          size = 10)))

#### Let's go to the next section: [Multi-objective optimisation of reservoir operating policy](3.b.%20Multi-objective%20optimisation%20of%20reservoir%20operating%20policy.ipynb)

### References 

Deb K. et al (2002) A fast and elitist multiobjective genetic algorithm: NSGA-II, IEEE Transactions on Evolutionary Computation, 6(2), 182-197, doi:10.1109/4235.996017.